In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
model_data = pd.DataFrame()

appeals_df = pd.read_csv('../data/zoning_board_appeals_tracker.csv')
voterdata_df = pd.read_csv('../data/VM2Uniform--MA--2021-01-19.tab', sep='\t', encoding='iso-8859-1', usecols=['Residence_Addresses_Zip', 'CommercialData_EstimatedHHIncomeAmount'])

zip_income_data = zip(voterdata_df['Residence_Addresses_Zip'], voterdata_df['CommercialData_EstimatedHHIncomeAmount'])

zip_to_mean_income = dict.fromkeys(voterdata_df['Residence_Addresses_Zip'], None)

for zipcode, income in zip_income_data:
    try:
        if zip_to_mean_income[zipcode] is None:
            zip_to_mean_income[zipcode] = []
        if type(income) == str:
            income = int(income[1:])
            zip_to_mean_income[zipcode].append(income)
    except:
        pass

for zipcode in zip_to_mean_income.keys():
    if zip_to_mean_income[zipcode] is not None:
        zip_to_mean_income[zipcode] = np.mean(zip_to_mean_income[zipcode])

/home/timevdo/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/timevdo/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [11]:
# Add model features
"""
We want
    appeal_type, status -- need to do indexing
    zip, ward, ever_deferred, num_deferrals -- include
    also include income of zip
"""
model_data[['zip', 'ward', 'ever_deferred', 'num_deferrals']] = appeals_df[['zip', 'ward', 'ever_deferred', 'num_deferrals']]
model_data['ever_deferred'] = model_data['ever_deferred'].fillna('N')
model_data['ever_deferred'] = model_data['ever_deferred'].replace('N', 0).replace('Y', 1)
model_data['num_deferrals'] = model_data['num_deferrals'].fillna(0)

appeal_list = list(set(appeals_df['appeal_type']))
status_list = list(set(appeals_df['status']))
model_data['appeal_type'] = appeals_df['appeal_type'].apply(lambda a : appeal_list.index(a))
model_data['status'] = appeals_df['status'].apply(lambda s : status_list.index(s))

model_data['income'] = appeals_df['zip'].apply(lambda z : zip_to_mean_income[z])
model_data['denied'] = appeals_df['decision'].apply(lambda d : 1 if d == 'DeniedPrej' or d == 'Denied' else 0)

model_data.head()

,zip,ward,ever_deferred,num_deferrals,appeal_type,status,income,denied
0,2125,13,0,0.0,0,1,85337.156606,0
1,2121,12,0,0.0,0,1,53875.984289,0
2,2128,1,0,0.0,0,1,77779.589967,0
3,2131,19,0,0.0,0,1,102257.050240,0
4,2130,19,0,0.0,0,1,120167.693612,0


In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(
        model_data.drop(['denied'], axis=1),
        model_data['denied'],
        test_size=1/4.0,
        random_state=0
    )

knn_model = BaggingClassifier(
    KNeighborsClassifier(n_neighbors=50, weights='distance'),
    n_estimators=50
).fit(X_train, Y_train)

Y_test_predictions = knn_model.predict(X_test)
print("Accuracy on testing set = ", accuracy_score(Y_test, Y_test_predictions))
print("RMSE on testing set = ", mean_squared_error(Y_test, Y_test_predictions))

Accuracy on testing set =  0.8805838124723574
RMSE on testing set =  0.11941618752764263


In [39]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, Y_train, Y_test = train_test_split(
        model_data.drop(['denied'], axis=1),
        model_data['denied'],
        test_size=1/4.0,
        random_state=0
    )

tree_model = RandomForestClassifier(max_features=None, n_estimators=1000).fit(X_train, Y_train)

Y_test_predictions = tree_model.predict(X_test)
print("Accuracy on testing set = ", accuracy_score(Y_test, Y_test_predictions))
print("RMSE on testing set = ", mean_squared_error(Y_test, Y_test_predictions))

Accuracy on testing set =  0.8841220698805838
RMSE on testing set =  0.11587793011941619
